In [1]:
import numpy as np
from PIL import Image
from sklearn.cluster import dbscan
from sklearn.preprocessing import MinMaxScaler
import cv2

from mmdet.models.utils.gaussian_target import get_local_maximum, get_topk_from_heatmap, transpose_and_gather_feat
from soft_nms import py_cpu_softnms

from utils import *
from xml_to_np import *

In [2]:
imgs = ( "/home/aiarhipov/datasets/WTW-dataset/test/images/0a3d89686e0b7989206f8c4b49c7edce7e8dd9a9.jpg", "/home/aiarhipov/datasets/WTW-dataset/test/images/0abd8c28799d176daf5839a227811b035fbf10a3.jpg", "/home/aiarhipov/datasets/WTW-dataset/test/images/0af4d29cbac7c16ef198e5f3789d984e4c3891bd.jpg", "/home/aiarhipov/datasets/WTW-dataset/test/images/0aNtiNtRRLqEZ9y6PuShtAAAACMAAQED.jpg", "/home/aiarhipov/datasets/WTW-dataset/test/images/0b5801be90efc3df0494483f1ef45cdd830252b6.jpg", "/home/aiarhipov/datasets/WTW-dataset/test/images/0c38706d33642e2c68389688009a4c3958601299.jpg", "/home/aiarhipov/datasets/WTW-dataset/test/images/000cce9ca593055d4618466e823e6d7c.jpg", "/home/aiarhipov/datasets/WTW-dataset/test/images/0cf3a942d5884587e9e16e82d258ee42e31f125a.jpg", "/home/aiarhipov/datasets/WTW-dataset/test/images/0d04e99791f1485159ce571027afc8383d8d09de.jpg", "/home/aiarhipov/datasets/WTW-dataset/test/images/0d60a677b078409f89cb85cd518f196fb77a6e07.jpg", "/home/aiarhipov/datasets/WTW-dataset/test/images/000d210d0f9b4101af2006b4aab33c42.jpg", "/home/aiarhipov/datasets/WTW-dataset/test/images/0d995a743dccbda88af4955cd07ab5bf3b8b663e.jpg", "/home/aiarhipov/datasets/WTW-dataset/test/images/000d9580eb97d677dbfdf6016b619280.jpg", "/home/aiarhipov/datasets/WTW-dataset/test/images/0dbcc99cff7d2dd79e43adfaa183997bd423cd6d.jpg", "/home/aiarhipov/datasets/WTW-dataset/test/images/0ddd7f772a4d90d9accfab5504076af1c2bfdf5a.jpg", "/home/aiarhipov/datasets/WTW-dataset/test/images/0e002b42313d92a48cbce1df0a79bbd859782a15.jpg")
checkpoint_file="/home/aiarhipov/centernet/exps/32_quad_long/latest.pth"
config_file="/home/aiarhipov/centernet/exps/32_quad_long/config.py"

cfg = Config.fromfile(config_file)

set_random_seed(0, deterministic=False)
model = build_detector(cfg.model, train_cfg=cfg.model.test_cfg)
model.CLASSES = ("box",)

model = init_detector(config_file, checkpoint_file, device=f"cuda:{get_free_gpu()}")
        
for img in imgs:
    img_path = img
    xml_path = f"/home/aiarhipov/datasets/WTW-dataset/test/xml/{img_path.split('/')[-1][:-4]}.xml"
    
    img = cv2.imread(img_path)
    quads = detect_quads(img_path, model, integer=False)
    boxes = np.vstack([(quads[:, 0] + quads[:, 6]) / 2, 
                       (quads[:, 1] + quads[:, 3]) / 2, 
                       (quads[:, 2] + quads[:, 4]) / 2, 
                       (quads[:, 5] + quads[:, 7]) / 2]).T
    
    nms_indexes = py_cpu_softnms(boxes.astype(int), quads[:, 8], thresh=0.475, method=2)
    quads = quads.astype(int)
    quads = quads[nms_indexes]

    gt_boxes = all_xml_to_np(xml_path)
    gt_boxes = gt_boxes.astype(int)
    # for quad in gt_boxes:
    #     cv2.line(img, (quad[0], quad[1]), (quad[2], quad[3]), (255, 0, 0), thickness=3)
    #     cv2.line(img, (quad[2], quad[3]), (quad[4], quad[5]), (255, 0, 0), thickness=3)
    #     cv2.line(img, (quad[4], quad[5]), (quad[6], quad[7]), (255, 0, 0), thickness=3)
    #     cv2.line(img, (quad[6], quad[7]), (quad[0], quad[1]), (255, 0, 0), thickness=3)
    
    
    center_heatmap_preds, offset_preds, c2v_pred, v2c_pred = heads_output(img_path, model)
    height, width = center_heatmap_preds[0].shape[2:]
    inp_h, inp_w = img.shape[:2]

    center_heatmap_pred = get_local_maximum(center_heatmap_preds[0][:, 1:2, ...], kernel=1)

    *batch_dets, topk_ys, topk_xs = get_topk_from_heatmap(center_heatmap_pred, k=1000)
    batch_scores, batch_index, batch_topk_labels = batch_dets
    offset = transpose_and_gather_feat(offset_preds[0], batch_index)
    # print(topk_xs, topk_ys)

    topk_xs = topk_xs + offset[..., 0]
    topk_ys = topk_ys + offset[..., 1]
    # print(topk_xs, topk_ys)
    for x, y in zip(topk_xs[0].cpu().detach().numpy(), topk_ys[0].cpu().detach().numpy()):
        cv2.circle(img, (int(x * (inp_w / width)), int(y * (inp_h / height))), radius=1, color=(0, 0, 255), thickness=-1)
        
    

    vertexes = np.vstack([topk_xs[0].cpu().detach().numpy(), topk_ys[0].cpu().detach().numpy()]).T
    scaler = MinMaxScaler()
    clusters = dbscan(scaler.fit_transform(vertexes), eps=0.02, min_samples=2)
    index, labels = clusters
    for label in np.unique(labels):
        if label != -1:
            x, y = vertexes[labels == label].mean(axis=0)
            cv2.circle(img, (int(x * (inp_w / width)), int(y * (inp_h / height))), radius=2, color=(255, 0, 255), thickness=-1)
            
    for quad in quads:
        cv2.line(img, (quad[0], quad[1]), (quad[2], quad[3]), (0, 255, 0), thickness=1)
        cv2.line(img, (quad[2], quad[3]), (quad[4], quad[5]), (0, 255, 0), thickness=1)
        cv2.line(img, (quad[4], quad[5]), (quad[6], quad[7]), (0, 255, 0), thickness=1)
        cv2.line(img, (quad[6], quad[7]), (quad[0], quad[1]), (0, 255, 0), thickness=1)
        
        cv2.circle(img, (quad[0], quad[1]), radius=2, color=(0, 255, 0), thickness=-1)
        cv2.circle(img, (quad[2], quad[3]), radius=2, color=(0, 255, 0), thickness=-1)
        cv2.circle(img, (quad[4], quad[5]), radius=2, color=(0, 255, 0), thickness=-1)
        cv2.circle(img, (quad[6], quad[7]), radius=2, color=(0, 255, 0), thickness=-1)
    # for quad in gt_boxes:
    #     cv2.circle(img, (quad[0], quad[1]), radius=2, color=(255, 0, 0), thickness=-1)
    #     cv2.circle(img, (quad[2], quad[3]), radius=2, color=(255, 0, 0), thickness=-1)
    #     cv2.circle(img, (quad[4], quad[5]), radius=2, color=(255, 0, 0), thickness=-1)
    #     cv2.circle(img, (quad[6], quad[7]), radius=2, color=(255, 0, 0), thickness=-1)
    res = Image.fromarray(img)
    res.save(f"""/home/aiarhipov/centernet/imgs/dbscan_test/{img_path.split('/')[-1]}""")

/home/aiarhipov/mmdetection/mmdet/models/builder.py:51: UserWarning: train_cfg and test_cfg is deprecated, please specify them in model
  warnings.warn(


Using GPU=4


2023-06-19 17:44:44,513 - root - INFO - ModulatedDeformConvPack neck.deconv_layers.0.conv is upgraded to version 2.
2023-06-19 17:44:44,516 - root - INFO - ModulatedDeformConvPack neck.deconv_layers.2.conv is upgraded to version 2.
2023-06-19 17:44:44,519 - root - INFO - ModulatedDeformConvPack neck.deconv_layers.4.conv is upgraded to version 2.


load checkpoint from local path: /home/aiarhipov/centernet/exps/32_quad_long/latest.pth


/home/aiarhipov/centernet/post-processing/soft_nms.py:73: RuntimeWarning: invalid value encountered in divide
  ovr = inter / (areas[i] + areas[pos:] - inter)
